# Task09 图神经网络的表示能力

## 1 知识回顾

- GNN通用框架包括邻域信息转换、邻域信息集合、跨层连接、输入图和特征的增广、学习目标
- GNN训练步骤：拆分数据集、预测头、损失函数、模型评估

## 2 GNN的表示能力

### 2.1 GNN基础概念

- 图神经网络核心思想：节点使用神经网络聚合来自其本地网络邻域的信息生成节点嵌入

- 单个GNN层：
    - 信息计算：$m_u^{(l)} = \text{MSG}^{(l)} (h_u^{(l - 1)})$
    - 信息聚合：$h_v^{(l)} = \text{AGG}^{(l)} (\{ m_u^{(l)}, u \in N(v) \})$

- GNN模型：
    - GCN：均值池化 + 线性 + ReLU非线性
    - GraphSAGE：MLP + 最大池化

### 2.2 解决GNN无法区分计算图根节点的问题

- GNN的表达能力：等价于区分计算图根节点Embedding的能力

- 思路讨论：
    - 一般来说，不同的局部邻域定义不同的计算图，计算图与每个节点周围的根节点子树结构相同。
    - 实现思路：使用单射函数，将不同的根子树映射到不同的节点嵌入中，先获得树的单个级别的结构，然后利用递归算法得到树的整个结构
    - 如果GNN聚合的每一步都可以完全保留相邻信息，则生成的节点嵌入可以区分不同的有根子树
    - 理想GNN：不同的计算图根节点输出不同的Embedding

## 3 设计最具表现力的GNN

- 设计前提：
    - GNN的表达能力可以通过使用邻域聚合函数来表征
    - 更具表现力的聚合函数会使得GNN更具表现力
    - 单射聚合函数可以得到最具表现力的GNN

- 讨论：
    - 邻域聚合可以抽象为多重集合（具有重复元素的集和）上的函数
    - GCN的聚合函数（平均池化层）无法区分颜色比例相同的不同超集
    - GraphSAGE的聚合函数无法区分不同的超集与同一集合下的不同颜色
    - GCN和GraphSAGE并不是最强大的GNN

- 设计最具表现力的GNN
    - 通过在多重集合上设计单射邻域聚合函数来实现
    - 任何单射多重集合函数都能表示为$\displaystyle \Phi \left( \sum_{x \in S} f(x) \right )$
    - 万能近似定理：在具有一个隐藏层的MLP模型中，当隐藏层的维度足够大，且使用非线性函数可以将任何连续函数近似到任意精度

- 图同构网络（Graph Isomorphism Network，GIN）：
$$
\text{MLP}_{\Phi} \left ( \sum_{x \in S} \text{MLP}_f (x) \right )
$$

- GIN算法步骤：
    1. 初始化每个节点的颜色$c^{(0)}(v)$
    2. 使用如下公式，对节点的颜色进行迭代更新
    $$
    c^{(k+1)}(v) = \text{HASH} \left( c^{(k)}(v), \{c^{(k)}(u)\}_{u \in N(v)} \right)
    $$
    3. 迭代$K$步后，得到稳定的节点颜色$c^{(K)}(v)$

- GIN模型：
$$
\text{MLP}_{\Phi} \left( (1 + \epsilon) \cdot \text{MLP}_f (c^{(k)}(v)) + \sum_{u \in N(v)} \text{MLP}_f (c^{(k)}(u)) \right)
$$

- 如果输入特征$c^{(0)}(v)$表示为独热向量，那么直接求和的函数就是单射，GIN卷积层可以表示为：
$$
\text{GINConv}\left( c^{(k)}, \{ c^{(k)}(u)\}_{u \in N(v)} \right) = \text{MLP}_{\Phi} \left( (1 + \epsilon) \cdot c^{(k)}(v) + \sum_{u \in N(v)} c^{(k)}(u) \right)
$$

- GIN可以理解为WL graph Kernel的可微神经版本

- GIN相对于WL图内核的优点：
    - 节点嵌入是低维的；因此，它们可以捕获不同节点的细粒度相似性
    - 可以根据下游任务学习优化

## 4 问题排查

### 4.1 通用解决方案

- 数据预处理：特征标准化处理
- 优化器：使用ADAM优化器
- 激活函数：ReLU通常效果很良好，可使用LeakyReLU、PReLU等其他激活函数，输出层没有激活函数，每一层包含偏置项
- 嵌入维度：通常选择32、64和128

### 4.2 调试深度网络

- Debug问题：损失/准确值在训练时未收敛
    - 检查pipeline
    - 调整学习率等超参数
    - 注意权重参数初始化
    - 仔细观察损失函数

- 模型开发：
    - 在训练集上存在过拟合情况
    - 检查loss曲线

## 5 本章总结

本次任务，主要介绍了图神经网络的表示能力，包括：

- GNN表示能力：GNN的表达能力等价于区分计算图根节点Embedding的能力
- 图同构网络（GIN）：GIN可以理解为WL graph Kernel的可微神经版本
- 问题排查：数据预处理、优化器、激活函数、调整嵌入维度、Debug深度网络的方法